In [ ]:
import pandas as pd

In [ ]:
file_path = 'terminos_definiciones.xlsx'  # Reemplaza con la ruta a tu archivo
df = pd.read_excel(file_path)
para_lematizar = df.columns[0]

In [ ]:
import pandas as pd

# Ruta al archivo de Excel
archivo_excel = 'terminos_definiciones.xlsx'

# Leer la hoja llamada 'Hoja1', usando la primera fila como cabecera, leyendo solo columnas A a C, saltando la primera fila y leyendo 10 filas.
df = pd.read_excel(archivo_excel, sheet_name='Hoja1', header=0, usecols="A:B", skiprows=None, nrows=None)

# Mostrar las primeras filas del DataFrame
print(df.head())


                          Término  \
0  carga viral indetectable (vih)   
1  carga viral indetectable (vih)   
2  carga viral indetectable (vih)   
3                       cisexismo   
4                        don juan   

                                        Definiciones  
0  Se refiere a una persona que vive con el diagn...  
1  Cuando tienes un virus es tu cuerpo, pero está...  
2                                       Asintomático  
3              Estar en contra de las personas trans  
4  Hombre que se relaciona de manera sexual con m...  


In [ ]:
terminos = set()
for termino in df['Término']:
    terminos.add(termino)

In [ ]:
import requests
def lematiza(fila):
    cadena_entrada = fila["Definiciones"]
    archivo = open("archivo.txt","w",encoding="utf-8")
    archivo.write(cadena_entrada)
    archivo.close()
    files = {'file': open("archivo.txt", 'rb')}
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}
    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
    #Convertir de formato json
    obj = r.json()
    #Ejemplo, obtener todos los lemas
    #print(obj)
    cadena = ""
    for sentence in obj:
      for word in sentence:
        cadena = cadena + " "+ word["lemma"]
    return pd.Series([cadena])

In [ ]:
df[['Def_lema']] = df.apply(lematiza, axis=1)

In [ ]:
df.to_excel('termino_definiciones_lematizados.xlsx', index=False)

In [ ]:
import re
def limpiar_espacios_multipalabras(cadena):
    # Eliminar espacios después de paréntesis de apertura y antes de paréntesis de cierre
    cadena = re.sub(r'\(\s+', '(', cadena)  # Quita espacios después de '('
    cadena = re.sub(r'\s+\)', ')', cadena)  # Quita espacios antes de ')'
    # Eliminar espacios al inicio y al final de la cadena
    cadena = cadena.strip()
    cadena = cadena.replace(" ", "_")
    return cadena

In [ ]:
import os

terminos = {}
for archivo in os.listdir("terminos"):
    original = open("terminos/"+archivo)
    cadena_original = original.readline()
    traducido = open("terminos_lematizados/"+archivo)
    cadena_traducida = traducido.readline()
    print(cadena_original+":"+limpiar_espacios_multipalabras(cadena_traducida))
    terminos[cadena_original] = limpiar_espacios_multipalabras(cadena_traducida)

reorientación sexual:reorientación_sexual
hermafroditismo verdadero:hermafroditismo_verdadero
vespasiano (urinario):vespasiano_(urinario)
pene duro:pene_duro
macrofilia:macrofilia
histeria:histeria
caballeroso:caballeroso
esposo:esposo
panocha:panocha
familia homoparental:familia_homoparental
poliginia:poliginia
joven:joven
lesbianismo:lesbianismo
antipedofilia:antipedofilia
machismo:machismo
flechazo:flechazo
oligospermia:oligospermia
iglesias cristianas inclusivas:iglesia_cristiano_inclusivo
terapia sexual:terapia_sexual
testículo:testículo
capitalismo rosa:capitalismo_rosa
apegar:apegar
transfobia:transfobia
esterilidad:esterilidad
higiene:higiene
frottage (sexualidad):frottage_(sexualidad)
desnudo:desnudo
crimen de honor:crimen_de_honor
sexo anónimo:sexo_anónimo
mutilación genital:mutilación_genital
eclampsia:eclampsia
circuncisión:circuncisión
recto:recto
abuso:abuso
manipulación:manipulación
sexo sin penetración:sexo_sin_penetración
poliamor:poliamor
erección:erección
deseo:desea

In [ ]:
def lema_termino(fila):
    term = fila["Término"]
    nuevo_term = terminos[term]
    return pd.Series([nuevo_term])


In [ ]:
df[['term_lema']] = df.apply(lema_termino, axis=1)

In [ ]:
df.to_excel('termino_definiciones_lematizados.xlsx', index=False)

In [2]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
vacias = stopwords.words('spanish')nlp = spacy.load('es_core_news_sm')

SyntaxError: invalid syntax (<ipython-input-2-c4ab48bafb64>, line 7)

In [1]:
import spacy
nlp = spacy.load('es_core_news_sm')

OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
def remueve_stop(fila):
    contenido = fila["Def_lema"]
    contenido = contenido.strip()
    patron = r'[^\w\s]'
    contenido = re.sub(patron, '',contenido)
    doc = nlp(contenido)
    palabras_sin_stopwords = [token.text for token in doc if not token.is_stop]
    sinstop = ' '.join(palabras_sin_stopwords)
    token = word_tokenize(sinstop)
    for palabra in vacias:
        token = [s for s in token if s != palabra]
    token = [s for s in token if s != " "]
    cadena = " ".join(token)
    return pd.Series([cadena])

In [ ]:
df[['def_lema_limpia']] = df.apply(remueve_stop, axis=1)

In [ ]:
df.to_excel('termino_definiciones_lematizados.xlsx', index=False)